In [6]:
#Covid_data Part

In [1]:
 %matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# variable to hold csv
#covid case data by state including testing
covidtrack_to_load = "Resources/all-states-history 10-26-2020.csv"
#Data from NYT COVID19 state data API
NYtimes_to_load = "Resources/data from NYT 10-20-2020.csv"
#data recording population of US states taken in 2019
populationfile = 'Resources/pop_est19.csv'
#matches state names to their abbreviations
keyfile = 'Resources/state_abbr list.csv'
#creates dataframe for state abbreviation keys
keydf = pd.read_csv(keyfile)
#creates dataframe for state population data
populationdf = pd.read_csv(populationfile)
#makes state name index of populationdf to support merging with other datasets
populationdf = populationdf.set_index(["NAME"])
# reads covidtrack data into dataframe
covidtrack_data = pd.read_csv(covidtrack_to_load)
#loads NYT COVID19 api data into dataframe
NYtimes_data = pd.read_csv(NYtimes_to_load)
#displays covidtrack data
covidtrack_data.head()

,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2020-10-26,AK,A,68.0,68.0,0,NaN,NaN,NaN,50.0,...,570844.0,10875,NaN,NaN,NaN,NaN,NaN,0,570844.0,10875
1,2020-10-26,AL,A,2866.0,2680.0,0,186.0,19974.0,19974.0,967.0,...,1307694.0,5028,NaN,NaN,80313.0,NaN,NaN,0,1307694.0,5028
2,2020-10-26,AR,A+,1833.0,1676.0,21,157.0,6768.0,6768.0,634.0,...,1294845.0,6845,NaN,21856.0,NaN,47404.0,NaN,0,1294845.0,6845
3,2020-10-26,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,1616.0,0,NaN,NaN,NaN,NaN,NaN,0,1616.0,0
4,2020-10-26,AZ,A+,5875.0,5587.0,1,288.0,21097.0,21097.0,837.0,...,1714801.0,8394,314045.0,NaN,NaN,NaN,1714801.0,8394,NaN,0


In [3]:
#loads csv containing state coordinate data for creating a heatmap plot
#loads data into df
state_coord_df = pd.read_csv('Resources/state_geocoord.csv', header=0)
state_coord_df.head()
state_coord_df.rename(columns = {'name':'States'}, inplace = True)
state_coord_df


,state,latitude,longitude,States
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California
5,CO,39.550051,-105.782067,Colorado
6,CT,41.603221,-73.087749,Connecticut
7,DC,38.905985,-77.033418,District of Columbia
8,DE,38.910832,-75.527670,Delaware
9,FL,27.664827,-81.515754,Florida


In [57]:
abbr_df = state_coord_df.drop(['latitude','longitude'], axis=1)
abbr_df.head()

,state,States
0,AK,Alaska
1,AL,Alabama
2,AR,Arkansas
3,AZ,Arizona
4,CA,California


In [4]:
#takes only relevant columns from covidtrack_data and loads into new df
reduced_covidtrack_data = pd.DataFrame(covidtrack_data, columns = ["date", "state", "negative", "negativeIncrease","positive","positiveIncrease","totalTestResults","totalTestResultsIncrease","deathIncrease"])
reduced_covidtrack_data["date"]=pd.to_datetime(reduced_covidtrack_data["date"])

In [5]:
#fills NA/NanN values with 0 
final_data = reduced_covidtrack_data
final_data['negative'] = final_data['negative'].fillna(0)
final_data['negativeIncrease'] = final_data['negativeIncrease'].fillna(0)
final_data['positive'] = final_data['positive'].fillna(0)
final_data['positiveIncrease'] = final_data['positiveIncrease'].fillna(0)
final_data['totalTestResults'] = final_data['totalTestResults'].fillna(0)
final_data['totalTestResultsIncrease'] = final_data['totalTestResultsIncrease'].fillna(0)
final_data['deathIncrease'] = final_data['deathIncrease'].fillna(0)
# final_data["total count"]=final_data["positive"]+final_data["negative"]

final_data

,date,state,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,deathIncrease
0,2020-10-26,AK,556431.0,10526,14413.0,349,570844.0,10875,0
1,2020-10-26,AL,1148993.0,4208,185322.0,967,1307694.0,5028,0
2,2020-10-26,AR,1195435.0,6315,106727.0,612,1294845.0,6845,21
3,2020-10-26,AS,1616.0,0,0.0,0,1616.0,0,0
4,2020-10-26,AZ,1481321.0,7615,238964.0,801,1714801.0,8394,1
...,...,...,...,...,...,...,...,...,...
13264,2020-01-24,WA,0.0,0,0.0,0,0.0,0,0
13265,2020-01-23,MA,0.0,0,0.0,0,2.0,1,0
13266,2020-01-23,WA,0.0,0,0.0,0,0.0,0,0
13267,2020-01-22,MA,0.0,0,0.0,0,1.0,0,0


In [6]:
# renamed_df = final_data.rename(columns={"state_x":"States"})
final_data_lng_lat = pd.merge(final_data, state_coord_df, on="state", how="left")
final_data_lng_lat

,date,state,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,deathIncrease,latitude,longitude,States
0,2020-10-26,AK,556431.0,10526,14413.0,349,570844.0,10875,0,63.588753,-154.493062,Alaska
1,2020-10-26,AL,1148993.0,4208,185322.0,967,1307694.0,5028,0,32.318231,-86.902298,Alabama
2,2020-10-26,AR,1195435.0,6315,106727.0,612,1294845.0,6845,21,35.201050,-91.831833,Arkansas
3,2020-10-26,AS,1616.0,0,0.0,0,1616.0,0,0,NaN,NaN,NaN
4,2020-10-26,AZ,1481321.0,7615,238964.0,801,1714801.0,8394,1,34.048928,-111.093731,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...
13264,2020-01-24,WA,0.0,0,0.0,0,0.0,0,0,47.751074,-120.740139,Washington
13265,2020-01-23,MA,0.0,0,0.0,0,2.0,1,0,42.407211,-71.382437,Massachusetts
13266,2020-01-23,WA,0.0,0,0.0,0,0.0,0,0,47.751074,-120.740139,Washington
13267,2020-01-22,MA,0.0,0,0.0,0,1.0,0,0,42.407211,-71.382437,Massachusetts


In [7]:
#creates summary data of covid statistics by state name
data_group_by = final_data_lng_lat.groupby(["States"])

Total_positive = data_group_by["positiveIncrease"].sum()
Total_negaitive = data_group_by["negativeIncrease"].sum()
Total_test = data_group_by["totalTestResultsIncrease"].sum()
Total_Deaths = data_group_by['deathIncrease'].sum()
latitude= data_group_by['latitude'].mean()
longitude= data_group_by['longitude'].mean()

state_covid_totals = pd.DataFrame({"Total positive":Total_positive,
                                  "Total negaitive":Total_negaitive,
                                  "Total test":Total_test,
                                  "Total Deaths":Total_Deaths})
#                                   "Latitude":latitude,
#                                   "Longitude":longitude
                                 
#adds state population data into dataframe for per capita calculations
state_covid_totals = state_covid_totals.join(populationdf)
state_covid_totals.drop(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'POPEST18PLUS2019', 'PCNT_POPEST18PLUS' ], axis=1, inplace=True)   
state_covid_totals.rename(columns = {'POPESTIMATE2019':'Population'}, inplace = True)
state_covid_totals.rename(columns = {'Total negaitive':'Total negative'}, inplace = True)


state_covid_totals

,Total positive,Total negative,Total test,Total Deaths,Population
States,,,,,
Alabama,185322,1148993,1307694,2866,4903185.0
Alaska,14413,556423,570836,68,731545.0
Arizona,238962,1481294,1714772,5875,7278717.0
Arkansas,106727,1195429,1294839,1833,3017804.0
California,900957,17081357,17982314,17357,39512223.0
Colorado,95087,1061126,1842254,2076,5758736.0
Connecticut,68099,2139838,2207937,4589,3565287.0
Delaware,24168,317893,342061,685,973764.0
District of Columbia,16812,481372,498184,642,705749.0


In [8]:
#Political part

In [9]:
#files to load in for dfs
geopolitics_file = "Resources/2016_US_County_Level_Presidential_Results.csv"
geopolitics_df = pd.read_csv(geopolitics_file)

merged_df_geo =pd.merge(geopolitics_df,keydf,on="state_abbr",how="left")
geopolitics_df_duplicates_removed = pd.DataFrame.drop_duplicates(merged_df_geo)
clean_politics_2016_df = pd.DataFrame(geopolitics_df_duplicates_removed)
clean_politics_2016_df

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,state
0,0,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2013,Alaska
1,1,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2016,Alaska
2,2,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2020,Alaska
3,3,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2050,Alaska
4,4,93003.0,130413.0,246588.0,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2060,Alaska
...,...,...,...,...,...,...,...,...,...,...,...,...
3136,3136,3233.0,12153.0,16661.0,0.194046,0.729428,"8,920",53.54%,WY,Sweetwater County,56037,Wyoming
3137,3137,7313.0,3920.0,12176.0,0.600608,0.321945,"3,393",27.87%,WY,Teton County,56039,Wyoming
3138,3138,1202.0,6154.0,8053.0,0.149261,0.764187,"4,952",61.49%,WY,Uinta County,56041,Wyoming
3139,3139,532.0,2911.0,3715.0,0.143203,0.783580,"2,379",64.04%,WY,Washakie County,56043,Wyoming


In [10]:
#creates a political preference score for each state based on republican votes / democrat votes
clean_politics_2016_df["poli_pref"] = (clean_politics_2016_df["votes_dem"]/clean_politics_2016_df["votes_gop"]).round(2)
clean_politics_2016_df.rename(columns = {'state':'States'}, inplace = True)
clean_politics_2016_df.tail()

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,States,poli_pref
3136,3136,3233.0,12153.0,16661.0,0.194046,0.729428,"8,920",53.54%,WY,Sweetwater County,56037,Wyoming,0.27
3137,3137,7313.0,3920.0,12176.0,0.600608,0.321945,"3,393",27.87%,WY,Teton County,56039,Wyoming,1.87
3138,3138,1202.0,6154.0,8053.0,0.149261,0.764187,"4,952",61.49%,WY,Uinta County,56041,Wyoming,0.20
3139,3139,532.0,2911.0,3715.0,0.143203,0.783580,"2,379",64.04%,WY,Washakie County,56043,Wyoming,0.18
3140,3140,294.0,2898.0,3334.0,0.088182,0.869226,"2,604",78.10%,WY,Weston County,56045,Wyoming,0.10


In [32]:
#groups county vote data by state to create a summary dataframe of party preference by state
data_group_by=clean_politics_2016_df.groupby(clean_politics_2016_df['States'])
votes_dem = data_group_by['votes_dem'].sum()
votes_gop = data_group_by['votes_gop'].sum()
poli_pref = data_group_by['poli_pref'].mean()
test = votes_dem/votes_gop
state_politics_df= pd.DataFrame({'votes_dem':votes_dem,
                  'votes_gop':votes_gop,
                  'Party Score':test})


state_politics_df

,votes_dem,votes_gop,Party Score
States,,,
Alabama,718084.0,1306925.0,0.549445
Alaska,2697087.0,3781977.0,0.713142
Arizona,936250.0,1021154.0,0.916855
Arkansas,378729.0,677904.0,0.558676
California,7362490.0,3916209.0,1.880004
Colorado,1212209.0,1137455.0,1.065720
Connecticut,884432.0,668266.0,1.323473
Delaware,235581.0,185103.0,1.272702
District of Columbia,260223.0,11553.0,22.524279


In [38]:
#joins covid data to state politics data
covid_state_politics = state_covid_totals.join(state_politics_df)
#calculates covid deaths per 1 M residents for each state (for later regression analysis)
covid_state_politics['Cases per 1 M'] = ((covid_state_politics['Total positive'])/covid_state_politics['Population'])*1000000
covid_state_politics['Deaths per 1 M'] = ((covid_state_politics['Total Deaths'])/covid_state_politics['Population'])*1000000
column_names = ["Total positive", "Total negative", "Total test", "Total Deaths", "Population", "Cases per 1 M", "Deaths per 1 M", "votes_dem", "votes_gop","Party Score"]
covid_state_politics = covid_state_politics.reindex(columns=column_names)
#"Party Score" column indicates political preference of state (<1 is Republican-preference, >1 Democrat Preference)
# covid_state_politics.rename(columns = {'test':'Party Score'}, inplace = True)
covid_state_politics = covid_state_politics.drop("District of Columbia", axis=0)
covid_state_politics = covid_state_politics.drop("Puerto Rico", axis=0)
covid_state_politics["Death Rate"]=covid_state_politics["Total Deaths"]/covid_state_politics["Total positive"]
covid_state_politics

,Total positive,Total negative,Total test,Total Deaths,Population,Cases per 1 M,Deaths per 1 M,votes_dem,votes_gop,Party Score,Death Rate
States,,,,,,,,,,,
Alabama,185322,1148993,1307694,2866,4903185.0,37796.248765,584.518022,718084.0,1306925.0,0.549445,0.015465
Alaska,14413,556423,570836,68,731545.0,19702.137257,92.953954,2697087.0,3781977.0,0.713142,0.004718
Arizona,238962,1481294,1714772,5875,7278717.0,32830.236428,807.147743,936250.0,1021154.0,0.916855,0.024585
Arkansas,106727,1195429,1294839,1833,3017804.0,35365.782536,607.395311,378729.0,677904.0,0.558676,0.017175
California,900957,17081357,17982314,17357,39512223.0,22801.982060,439.281789,7362490.0,3916209.0,1.880004,0.019265
Colorado,95087,1061126,1842254,2076,5758736.0,16511.783141,360.495775,1212209.0,1137455.0,1.065720,0.021833
Connecticut,68099,2139838,2207937,4589,3565287.0,19100.566097,1287.133406,884432.0,668266.0,1.323473,0.067387
Delaware,24168,317893,342061,685,973764.0,24819.155360,703.455868,235581.0,185103.0,1.272702,0.028343
Florida,782011,5202712,5984723,16652,21477737.0,36410.307101,775.314457,4485745.0,4605515.0,0.973994,0.021294


In [61]:
covid_state_politics = covid_state_politics.merge(abbr_df, on = 'States')
covid_state_politics.head()

,States,Total positive,Total negative,Total test,Total Deaths,Population,Cases per 1 M,Deaths per 1 M,votes_dem,votes_gop,Party Score,Death Rate,state
0,Alabama,185322,1148993,1307694,2866,4903185.0,37796.248765,584.518022,718084.0,1306925.0,0.549445,0.015465,AL
1,Alaska,14413,556423,570836,68,731545.0,19702.137257,92.953954,2697087.0,3781977.0,0.713142,0.004718,AK
2,Arizona,238962,1481294,1714772,5875,7278717.0,32830.236428,807.147743,936250.0,1021154.0,0.916855,0.024585,AZ
3,Arkansas,106727,1195429,1294839,1833,3017804.0,35365.782536,607.395311,378729.0,677904.0,0.558676,0.017175,AR
4,California,900957,17081357,17982314,17357,39512223.0,22801.982060,439.281789,7362490.0,3916209.0,1.880004,0.019265,CA


In [62]:
#outputs combined dataframe for matplotlib calculations and T testing
output_data = "Output/final_dataframe.csv"
covid_state_politics.to_csv(output_data)
clean_df.to_csv(mapping_data)